# Bitcoin API Examples

This notebook shows, among others, a modified version of the code from the book **"Mastering Bitcoin"**, by A. Antonopoulos. Since a Bitcoin node provides an RPC interface, it is possible to query information to the node sitting in our local network using several tools, as it is shown.

- `cURL`.
- `cURL` using Python `requests`.
- The library `python-bitcoincli`, which also uses the RPC.

Table of contents:

- [Setup](#setup)
- [CURL Interface](#curl-interface)
- [Python-BitcoinLib](#python-bitcoinlib)
- [Get All the Outputs of a Transaction](#get-all-the-outputs-of-a-transaction)
- [Get Total Value in All Transactions of a Block](#get-total-value-in-all-transactions-of-a-block)

## Setup

This notebook shows, among others, a modified version of the code from the book **"Mastering Bitcoin"**, by A. Antonopoulos. Since a Bitcoin node provides an RPC interface, it is possible to query information to the node sitting in our local network using several tools:

- `cURL`.
- `cURL` using Python `requests`.
- The library `python-bitcoincli`, which also uses the RPC.

But first, we need to set everything up:

- First, you need to set up a Bitcoin node. Check [`Bitcoing_Practical_Guide.md`](../bitcoin_practical/Bitcoing_Practical_Guide.md).
- Then, a `.env` file needs to be created, which contains the values of `RPC_USER`, `RPC_PASSWORD`, `BTC_NODE_IP` and `RPC_PORT`. Those values come from the node. See [`Bitcoin_Antonopoulos_Book.md`](./Bitcoin_Antonopoulos_Book.md).
- Finally, install the conda environment with `conda.yaml` and you should be ready to run these examples.

## CURL Interface

We can use `cURL` to send any `bitcoin-cli` command to our node via RPC:

```bash
# Command in the node
bitcoin-cli getblockchaininfo

# Equivalent cURL sent from another machine in the local network
curl --user $RPC_USER:$RPC_PASSWORD --data-binary '{"jsonrpc": "1.0", "id":"curltest", "method": "getblockchaininfo", "params": [] }' -H 'content-type: text/plain;' http://$BTC_NODE_IP:$RPC_PORT/
```

## CURL in Python via Requests

In [1]:
import os
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
# Retrieve environment variables
rpc_user = os.getenv('RPC_USER')
rpc_password = os.getenv('RPC_PASSWORD')
rpc_ip = os.getenv('BTC_NODE_IP')
rpc_port = os.getenv('RPC_PORT')

# Create the connection URL
# Username + PW are passed later in HTTPBasicAuth
rpc_url = f"http://{rpc_ip}:{rpc_port}"

In [3]:
# Create the payload for the RPC request
payload = {
    "jsonrpc": "1.0",
    "id": "curltest",
    # We would modify method and params depending on the command
    "method": "getblockchaininfo",
    "params": []
}

In [4]:
# Send the RPC request
try:
    response = requests.post(rpc_url, json=payload, auth=HTTPBasicAuth(rpc_user, rpc_password), headers={'Content-Type': 'text/plain'})
    response.raise_for_status()  # Raise an exception for HTTP errors
    info = response.json()
    # Retrieve the 'blocks' element from the info
    print(info['result']['blocks']) # 851989
except Exception as e:
    print(f"An error occurred: {e}")

869589


## Python-BitcoinLib

In [5]:
import os
from dotenv import load_dotenv
from bitcoin.rpc import RawProxy

# Load environment variables from .env file
load_dotenv()

True

In [6]:
# Retrieve environment variables
rpc_user = os.getenv('RPC_USER')
rpc_password = os.getenv('RPC_PASSWORD')
rpc_ip = os.getenv('BTC_NODE_IP')
rpc_port = os.getenv('RPC_PORT')

# Create the connection URL
# We pass username + PW here!
rpc_url = f"http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}"

In [7]:
# Create a connection to the Bitcoin Core node
try:
    p = RawProxy(service_url=rpc_url)
    # Run the getblockchaininfo command, store the resulting data in info
    info = p.getblockchaininfo()
    # Retrieve the 'blocks' element from the info
    print(info['blocks']) # 851989
except Exception as e:
    print(f"An error occurred: {e}")

869589


## Get All the Outputs of a Transaction

In [8]:
import os
from dotenv import load_dotenv
from bitcoin.rpc import RawProxy

# Load environment variables from .env file
load_dotenv()

True

In [9]:
# Retrieve environment variables
rpc_user = os.getenv('RPC_USER')
rpc_password = os.getenv('RPC_PASSWORD')
rpc_ip = os.getenv('BTC_NODE_IP')
rpc_port = os.getenv('RPC_PORT')

# Create the connection URL
# We pass username + PW here!
rpc_url = f"http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}"

In [10]:
# Create a connection to the Bitcoin Core node
try:
    p = RawProxy(service_url=rpc_url)
except Exception as e:
    print(f"An error occurred: {e}")

In [11]:
# Alice's transaction ID
txid = "466200308696215bbc949d5141a49a4138ecdfdfaa2a8029c1f9bcecd1f96177"

# First, retrieve the raw transaction in hex
raw_tx = p.getrawtransaction(txid)

# Decode the transaction hex into a JSON object
decoded_tx = p.decoderawtransaction(raw_tx)

# Retrieve each of the outputs from the transaction
for output in decoded_tx['vout']:
    print(output['scriptPubKey']['address'], output['value'])
    # bc1p8dqa4wjvnt890qmfws83te0v3qxzsfu7ul63kp7u56w8qc0qwp5qv995qn 0.00020000
    # bc1qwafvze0200nh9vkq4jmlf4sy0tn0ga5w0zpkpg 0.00075000

bc1p8dqa4wjvnt890qmfws83te0v3qxzsfu7ul63kp7u56w8qc0qwp5qv995qn 0.00020000
bc1qwafvze0200nh9vkq4jmlf4sy0tn0ga5w0zpkpg 0.00075000


## Get Total Value in All Transactions of a Block

In [18]:
import os
from dotenv import load_dotenv
from bitcoin.rpc import RawProxy
from tqdm import tqdm

# Load environment variables from .env file
load_dotenv()

True

In [19]:
# Retrieve environment variables
rpc_user = os.getenv('RPC_USER')
rpc_password = os.getenv('RPC_PASSWORD')
rpc_ip = os.getenv('BTC_NODE_IP')
rpc_port = os.getenv('RPC_PORT')

# Create the connection URL
# We pass username + PW here!
rpc_url = f"http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}"

In [20]:
# Create a connection to the Bitcoin Core node
try:
    p = RawProxy(service_url=rpc_url)
except Exception as e:
    print(f"An error occurred: {e}")

In [21]:
# The block height where Alice's transaction was recorded
blockheight = 775072

# Get the block hash of the block at the given height
blockhash = p.getblockhash(blockheight)

# Retrieve the block by its hash
block = p.getblock(blockhash)

# Element tx contains the list of all transaction IDs in the block
transactions = block['tx']

block_value = 0

# Iterate through each transaction ID in the block
for txid in tqdm(transactions):
#for txid in transactions:
    tx_value = 0
    # Retrieve the raw transaction by ID
    raw_tx = p.getrawtransaction(txid)
    # Decode the transaction
    decoded_tx = p.decoderawtransaction(raw_tx)
    # Iterate through each output in the transaction
    for output in decoded_tx['vout']:
        # Add up the value of each output
        tx_value = tx_value + output['value']

    # Add the value of this transaction to the total
    block_value = block_value + tx_value

print("Total value in block: ", block_value) # 5863.00566521

100%|██████████| 1966/1966 [02:24<00:00, 13.57it/s]

Total value in block:  5863.00566521
